In [1]:
import pandas as pd
import json
from datetime import datetime, timedelta
import warnings
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import matplotlib.dates as mdates
warnings.filterwarnings(action='ignore')
pd.set_option('display.max_columns', None)

# Read the "acwr-data"

In [2]:
def load_and_feature_engineering_acwr(file_path):
    # read the data 
    acwr = pd.read_csv(file_path)
    # transform the date format
    for i, val in enumerate(acwr['date']):
        acwr['date'][i] = datetime.utcfromtimestamp(val)
    
    # drop the wellness data
    try:
        acwr = acwr.drop(columns=
                 ["fatigue","hydration","mood","nutrition","readiness",
                  "sleepduration","sleepquality","soreness","stress","total"], axis=1)
    except:
        acwr = acwr
    
    # transform the date type
    acwr['formatteddate'] = pd.to_datetime(acwr['date'])
    acwr["formatteddate_dt"] = pd.to_datetime(acwr["formatteddate"],format="%d/%m/%Y")
    acwr['formatteddate'] = acwr["formatteddate_dt"].dt.date.astype(str)
    
    return acwr
    

In [3]:
file_path = "acwr_updated.csv"
acwr = load_and_feature_engineering_acwr(file_path)
acwr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5292 entries, 0 to 5291
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   acute             5292 non-null   float64       
 1   acwr              5292 non-null   float64       
 2   chronic           5292 non-null   float64       
 3   condition         5205 non-null   object        
 4   date              5292 non-null   object        
 5   duration          5292 non-null   int64         
 6   load              5292 non-null   int64         
 7   rpe               5292 non-null   int64         
 8   type              5205 non-null   object        
 9   player_id         5292 non-null   object        
 10  name              5292 non-null   object        
 11  formatteddate     5292 non-null   object        
 12  formatteddate_dt  5292 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(3), int64(3), object(6)
memory usage: 537.6+ KB


In [4]:
acwr[acwr['player_id'] == '5f0593d9c37cc79a7d6a6cae']

,acute,acwr,chronic,condition,date,duration,load,rpe,type,player_id,name,formatteddate,formatteddate_dt
8,390.000000,1.000000,390.000000,Artificial (Dry),2020-06-29 00:00:00,65,390,6,Practice (Field),5f0593d9c37cc79a7d6a6cae,Shu Sasaki,2020-06-29,2020-06-29 00:00:00
26,337.500000,0.898760,375.517241,Gym/Indoors,2020-06-30 00:00:00,45,180,4,Practice (Gym),5f0593d9c37cc79a7d6a6cae,Shu Sasaki,2020-06-30,2020-06-30 00:00:00
27,442.500000,1.093990,404.482759,Grass (Dry),2020-06-30 00:00:00,60,600,7,Practice (Field),5f0593d9c37cc79a7d6a6cae,Shu Sasaki,2020-06-30,2020-06-30 00:00:00
38,413.125000,1.049187,393.757432,Artificial (Dry),2020-07-01 00:00:00,80,640,8,Practice (Field),5f0593d9c37cc79a7d6a6cae,Shu Sasaki,2020-07-01,2020-07-01 00:00:00
56,366.093750,0.958062,382.118988,Gym/Indoors,2020-07-02 00:00:00,45,225,5,Practice (Gym),5f0593d9c37cc79a7d6a6cae,Shu Sasaki,2020-07-02,2020-07-02 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5221,307.189238,1.279513,240.082933,Sand,2022-01-26 06:07:15,75,525,7,Practice (Beach),5f0593d9c37cc79a7d6a6cae,Shu Sasaki,2022-01-26,2022-01-26 06:07:15
5236,305.391929,1.250504,244.215145,Track,2022-01-27 06:07:15,50,300,6,Quarantine - Run,5f0593d9c37cc79a7d6a6cae,Shu Sasaki,2022-01-27,2022-01-27 06:07:15
5251,349.043946,1.340023,260.476169,Sand,2022-01-28 06:07:15,60,480,8,Practice (Beach),5f0593d9c37cc79a7d6a6cae,Shu Sasaki,2022-01-28,2022-01-28 06:07:15
5267,243.502915,1.028447,236.767495,Sand,2022-01-31 05:59:59,55,385,7,Practice (Beach),5f0593d9c37cc79a7d6a6cae,Shu Sasaki,2022-01-31,2022-01-31 05:59:59


In [5]:
numerical = ['acute', 'acwr', 'chronic', 'duration', 'load','rpe']
categorical = ['condition', 'type'] 
na_cols = ["type","condition"]

In [6]:
acwr[acwr.isna().any(axis=1)]
#len(acwr[acwr.isna().any(axis=1)])
#35

,acute,acwr,chronic,condition,date,duration,load,rpe,type,player_id,name,formatteddate,formatteddate_dt
3147,100.028229,1.943563,51.466417,NaN,2021-08-31 02:10:29,60,300,5,NaN,5f0592e2239906121f930929,Kota Kawase,2021-08-31,2021-08-31 02:10:29
3148,364.859214,1.171410,311.470227,NaN,2021-08-31 02:10:29,90,540,6,NaN,5f059326c37cc79a7d6a6ca9,james Ha,2021-08-31,2021-08-31 02:10:29
3149,275.873053,1.687937,163.437967,NaN,2021-08-31 02:10:29,90,630,7,NaN,5f059355c37cc79a7d6a6caa,Jack Sealy,2021-08-31,2021-08-31 02:10:29
3150,427.874266,1.318486,324.519490,NaN,2021-08-31 02:10:29,90,810,9,NaN,5f059378239906121f93092e,Xavi Cheng,2021-08-31,2021-08-31 02:10:29
3151,412.975247,1.299609,317.768832,NaN,2021-08-31 02:10:29,90,810,9,NaN,5f059388239906121f93092f,Paco Chan,2021-08-31,2021-08-31 02:10:29
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4590,311.801311,1.032951,301.854833,NaN,2021-12-09 04:46:12,60,360,6,NaN,60f254fb854df5b9da1bac56,LucianoDa Silva,2021-12-09,2021-12-09 04:46:12
4591,340.585829,1.230432,276.801786,NaN,2021-12-09 04:46:12,60,420,7,NaN,6107d0a36f7b01ca9c0f6571,ChanKa Po,2021-12-09,2021-12-09 04:46:12
4592,319.912931,1.027532,311.341001,NaN,2021-12-09 04:46:12,60,360,6,NaN,61139a2f859c5efc737749ba,Ju Yingzhi,2021-12-09,2021-12-09 04:46:12
4593,294.441589,1.041749,282.641598,NaN,2021-12-09 04:46:12,60,360,6,NaN,611a41e057d5db652811c0f3,ChenChin Lung,2021-12-09,2021-12-09 04:46:12


In [7]:
#impute the missing 'condition' and 'type' with the most frequent values
acwr1 = acwr.fillna(acwr.mode().iloc[0])
acwr1

,acute,acwr,chronic,condition,date,duration,load,rpe,type,player_id,name,formatteddate,formatteddate_dt
0,180.000000,1.000000,180.000000,Artificial (Dry),2020-06-29 00:00:00,45,180,4,Practice (Field),5f014aa4b8bec1cf08f7ca0f,Zesh Rehman,2020-06-29,2020-06-29 00:00:00
1,325.000000,1.000000,325.000000,Artificial (Dry),2020-06-29 00:00:00,65,325,5,Practice (Field),5f0592cc239906121f930928,TszTo Choy,2020-06-29,2020-06-29 00:00:00
2,390.000000,1.000000,390.000000,Artificial (Dry),2020-06-29 00:00:00,65,390,6,Practice (Field),5f0592e2239906121f930929,Kota Kawase,2020-06-29,2020-06-29 00:00:00
3,325.000000,1.000000,325.000000,Artificial (Dry),2020-06-29 00:00:00,65,325,5,Practice (Field),5f059326c37cc79a7d6a6ca9,james Ha,2020-06-29,2020-06-29 00:00:00
4,195.000000,1.000000,195.000000,Artificial (Dry),2020-06-29 00:00:00,65,195,3,Practice (Field),5f059355c37cc79a7d6a6caa,Jack Sealy,2020-06-29,2020-06-29 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5287,87.924115,0.514831,170.782587,Mixed,2022-02-04 05:59:59,40,120,3,Quarantine - Run,60f254d8a1b3ec106cc401da,Khan Jahangir,2022-02-04,2022-02-04 05:59:59
5288,145.724642,0.652914,223.191242,Mixed,2022-02-04 05:59:59,40,240,6,Quarantine - Run,60f254fb854df5b9da1bac56,LucianoDa Silva,2022-02-04,2022-02-04 05:59:59
5289,134.414729,0.724831,185.442837,Mixed,2022-02-04 05:59:59,40,240,6,Quarantine - Run,6107d0a36f7b01ca9c0f6571,ChanKa Po,2022-02-04,2022-02-04 05:59:59
5290,91.835734,0.577736,158.958055,Mixed,2022-02-04 05:59:59,40,200,5,Quarantine - Run,61139a2f859c5efc737749ba,Ju Yingzhi,2022-02-04,2022-02-04 05:59:59


In [8]:
set(acwr[acwr["type"]=="Game"]["condition"])

{'Artificial (Dry)', 'Artificial (Wet)', 'Grass (Dry)', 'Grass (Wet)'}

In [9]:
set(acwr[acwr["type"]!="Game"]["condition"])

{'Artificial (Dry)',
 'Artificial (Wet)',
 'Grass (Dry)',
 'Grass (Wet)',
 'Gym/Indoors',
 'Hike',
 'Mixed',
 'Sand',
 'Track',
 nan}

In [10]:
players = set(acwr["player_id"])

In [11]:
len(players)

21

# Read the "injurcsv"

In [12]:
def load_and_feature_engineering_injury(file_path):
    injury = pd.read_csv(file_path)
    for i, val in enumerate(injury['Injury Date']):
        injury['Injury Date'][i] = datetime.utcfromtimestamp(val)
    for i, val in enumerate(injury['Return Date']):
        injury['Return Date'][i] = datetime.utcfromtimestamp(val)
    
    injury['Injury Date'] = pd.to_datetime(injury['Injury Date'])#.dt.date()
    injury['Return Date'] = pd.to_datetime(injury['Return Date'])

    injury['FormattedInjuryDate'] = injury['Injury Date'].dt.date.astype(str)
    injury['FormattedReturnDate'] = injury['Return Date'].dt.date.astype(str)

    injury['FormattedInjuryDate_dt'] = pd.to_datetime(injury['FormattedInjuryDate'])
    injury['FormattedReturnDate_dt'] = pd.to_datetime(injury['FormattedReturnDate']) 
                                       
                                       
    injury = injury.drop(['Diagnosis','Notes'], axis=1)
    chronic_injury = injury[injury['Onset'] != 'Acute']
    return injury, chronic_injury

In [13]:
file_path = "injury_updated.csv"
injury, chronic_injury = load_and_feature_engineering_injury(file_path)

In [14]:
injury[injury['u_id']=='5f0593d9c37cc79a7d6a6cae']

,Activity,Classification,Injury Date,Location,OSIICs,Onset,Past Injury,Reoccurance,Return Date,Side,Surgery,u_id,FormattedInjuryDate,FormattedReturnDate,FormattedInjuryDate_dt,FormattedReturnDate_dt
2,General Training,New Injury,2020-12-21 22:01:30,DTE - Domestic,QM1,Repetitive - Sudden Onset,no,NaN,2021-01-25 22:01:30,Right,no,5f0593d9c37cc79a7d6a6cae,2020-12-21,2021-01-25,2020-12-21,2021-01-25
3,General Training,Recurrent Injury,2021-01-25 22:06:39,DTE - Domestic,QM1,Repetitive - Sudden Onset,no,NaN,2021-02-07 22:06:39,Right,no,5f0593d9c37cc79a7d6a6cae,2021-01-25,2021-02-07,2021-01-25,2021-02-07
11,Not Related to Sports,New Injury,2021-04-10 06:21:11,Other,KCP,Acute,no,Primary Injury,2021-04-14 06:21:11,Right,no,5f0593d9c37cc79a7d6a6cae,2021-04-10,2021-04-14,2021-04-10,2021-04-14
23,General Training,Recurrent Injury,2021-08-05 10:57:42,DTE - Domestic,NaN,Repetitive - Gradual Onset,no,Primary Injury,2021-08-09 10:57:42,Right,no,5f0593d9c37cc79a7d6a6cae,2021-08-05,2021-08-09,2021-08-05,2021-08-09


In [15]:
print(injury.info())
print(chronic_injury.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41 entries, 0 to 40
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Activity                40 non-null     object        
 1   Classification          41 non-null     object        
 2   Injury Date             41 non-null     datetime64[ns]
 3   Location                39 non-null     object        
 4   OSIICs                  16 non-null     object        
 5   Onset                   41 non-null     object        
 6   Past Injury             41 non-null     object        
 7   Reoccurance             34 non-null     object        
 8   Return Date             41 non-null     datetime64[ns]
 9   Side                    38 non-null     object        
 10  Surgery                 41 non-null     object        
 11  u_id                    41 non-null     object        
 12  FormattedInjuryDate     41 non-null     object      

# Merge the acwr and injury data

In [16]:
outer_df = pd.merge(acwr1, injury,  
                        how='outer', left_on=['player_id','formatteddate'], right_on = ['u_id','FormattedInjuryDate'])

In [17]:
outer_df.shape

(5305, 29)

In [18]:
outer_df[outer_df['player_id'] == '5f0593d9c37cc79a7d6a6cae']

,acute,acwr,chronic,condition,date,duration,load,rpe,type,player_id,name,formatteddate,formatteddate_dt,Activity,Classification,Injury Date,Location,OSIICs,Onset,Past Injury,Reoccurance,Return Date,Side,Surgery,u_id,FormattedInjuryDate,FormattedReturnDate,FormattedInjuryDate_dt,FormattedReturnDate_dt
8,390.000000,1.000000,390.000000,Artificial (Dry),2020-06-29 00:00:00,65.0,390.0,6.0,Practice (Field),5f0593d9c37cc79a7d6a6cae,Shu Sasaki,2020-06-29,2020-06-29 00:00:00,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaT
26,337.500000,0.898760,375.517241,Gym/Indoors,2020-06-30 00:00:00,45.0,180.0,4.0,Practice (Gym),5f0593d9c37cc79a7d6a6cae,Shu Sasaki,2020-06-30,2020-06-30 00:00:00,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaT
27,442.500000,1.093990,404.482759,Grass (Dry),2020-06-30 00:00:00,60.0,600.0,7.0,Practice (Field),5f0593d9c37cc79a7d6a6cae,Shu Sasaki,2020-06-30,2020-06-30 00:00:00,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaT
38,413.125000,1.049187,393.757432,Artificial (Dry),2020-07-01 00:00:00,80.0,640.0,8.0,Practice (Field),5f0593d9c37cc79a7d6a6cae,Shu Sasaki,2020-07-01,2020-07-01 00:00:00,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaT
56,366.093750,0.958062,382.118988,Gym/Indoors,2020-07-02 00:00:00,45.0,225.0,5.0,Practice (Gym),5f0593d9c37cc79a7d6a6cae,Shu Sasaki,2020-07-02,2020-07-02 00:00:00,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5222,307.189238,1.279513,240.082933,Sand,2022-01-26 06:07:15,75.0,525.0,7.0,Practice (Beach),5f0593d9c37cc79a7d6a6cae,Shu Sasaki,2022-01-26,2022-01-26 06:07:15,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaT
5237,305.391929,1.250504,244.215145,Track,2022-01-27 06:07:15,50.0,300.0,6.0,Quarantine - Run,5f0593d9c37cc79a7d6a6cae,Shu Sasaki,2022-01-27,2022-01-27 06:07:15,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaT
5252,349.043946,1.340023,260.476169,Sand,2022-01-28 06:07:15,60.0,480.0,8.0,Practice (Beach),5f0593d9c37cc79a7d6a6cae,Shu Sasaki,2022-01-28,2022-01-28 06:07:15,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaT
5268,243.502915,1.028447,236.767495,Sand,2022-01-31 05:59:59,55.0,385.0,7.0,Practice (Beach),5f0593d9c37cc79a7d6a6cae,Shu Sasaki,2022-01-31,2022-01-31 05:59:59,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaT


In [19]:
def feature_engineering_merged(outer_df):
    outer_df['player_id'].fillna(outer_df['u_id'], inplace=True)
    outer_df['formatteddate'].fillna(outer_df['FormattedInjuryDate'], inplace=True)
    outer_df['formatteddate_dt'] = pd.to_datetime(outer_df['formatteddate'])

    fill_zero_cols = ["acute","acwr","chronic","duration","load","rpe"]
    fill_none_cols = ['condition','type']

    for col in outer_df.columns:
        if col in fill_zero_cols:
            outer_df[col] = outer_df[col].fillna(0)
        elif col in fill_none_cols:
            outer_df[col] = outer_df[col].fillna('None')
        else:
            outer_df[col] = outer_df[col]
            
    outer_df["injury?"] = outer_df.apply(lambda row: 1 if 
                                         ((row['player_id'] == row['u_id']) and (row['formatteddate'] == row['FormattedInjuryDate']))
                                         else 0, axis = 1)
    return outer_df

In [20]:
outer_df = feature_engineering_merged(outer_df)
outer_df

,acute,acwr,chronic,condition,date,duration,load,rpe,type,player_id,name,formatteddate,formatteddate_dt,Activity,Classification,Injury Date,Location,OSIICs,Onset,Past Injury,Reoccurance,Return Date,Side,Surgery,u_id,FormattedInjuryDate,FormattedReturnDate,FormattedInjuryDate_dt,FormattedReturnDate_dt,injury?
0,180.0,1.0,180.0,Artificial (Dry),2020-06-29,45.0,180.0,4.0,Practice (Field),5f014aa4b8bec1cf08f7ca0f,Zesh Rehman,2020-06-29,2020-06-29,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaT,0
1,325.0,1.0,325.0,Artificial (Dry),2020-06-29,65.0,325.0,5.0,Practice (Field),5f0592cc239906121f930928,TszTo Choy,2020-06-29,2020-06-29,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaT,0
2,390.0,1.0,390.0,Artificial (Dry),2020-06-29,65.0,390.0,6.0,Practice (Field),5f0592e2239906121f930929,Kota Kawase,2020-06-29,2020-06-29,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaT,0
3,325.0,1.0,325.0,Artificial (Dry),2020-06-29,65.0,325.0,5.0,Practice (Field),5f059326c37cc79a7d6a6ca9,james Ha,2020-06-29,2020-06-29,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaT,0
4,195.0,1.0,195.0,Artificial (Dry),2020-06-29,65.0,195.0,3.0,Practice (Field),5f059355c37cc79a7d6a6caa,Jack Sealy,2020-06-29,2020-06-29,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaT,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5300,0.0,0.0,0.0,None,NaT,0.0,0.0,0.0,None,5f0592e2239906121f930929,NaN,2021-08-04,2021-08-04,Other,Recurrent Injury,2021-08-04 00:52:10,Other,NaN,Repetitive - Gradual Onset,yes,Primary Injury,2021-08-04 00:52:10,Right,yes,5f0592e2239906121f930929,2021-08-04,2021-08-04,2021-08-04,2021-08-04,1
5301,0.0,0.0,0.0,None,NaT,0.0,0.0,0.0,None,5f0592e2239906121f930929,NaN,2021-08-04,2021-08-04,Other,Recurrent Injury,2021-08-04 00:52:14,Other,NaN,Repetitive - Gradual Onset,yes,Primary Injury,2021-08-04 00:52:14,Right,yes,5f0592e2239906121f930929,2021-08-04,2021-08-04,2021-08-04,2021-08-04,1
5302,0.0,0.0,0.0,None,NaT,0.0,0.0,0.0,None,5f0592e2239906121f930929,NaN,2021-08-04,2021-08-04,Other,Recurrent Injury,2021-08-04 00:52:31,Other,NaN,Repetitive - Gradual Onset,yes,Primary Injury,2021-08-04 00:52:31,Right,yes,5f0592e2239906121f930929,2021-08-04,2021-08-04,2021-08-04,2021-08-04,1
5303,0.0,0.0,0.0,None,NaT,0.0,0.0,0.0,None,5f0592e2239906121f930929,NaN,2021-08-04,2021-08-04,Other,Recurrent Injury,2021-08-04 00:52:45,Other,NaN,Repetitive - Gradual Onset,yes,Primary Injury,2021-08-04 00:52:45,Right,yes,5f0592e2239906121f930929,2021-08-04,2021-08-04,2021-08-04,2021-08-04,1


In [21]:
outer_df[outer_df['u_id']=='5f0592e2239906121f930929']

,acute,acwr,chronic,condition,date,duration,load,rpe,type,player_id,name,formatteddate,formatteddate_dt,Activity,Classification,Injury Date,Location,OSIICs,Onset,Past Injury,Reoccurance,Return Date,Side,Surgery,u_id,FormattedInjuryDate,FormattedReturnDate,FormattedInjuryDate_dt,FormattedReturnDate_dt,injury?
2024,467.745603,1.269344,368.493849,Grass (Wet),2021-02-27 15:34:55,95.0,950.0,10.0,Game,5f0592e2239906121f930929,Kota Kawase,2021-02-27,2021-02-27,Competition,Recurrent Injury,2021-02-27 03:35:51,Competition - Domestic,AA1,Repetitive - Sudden Onset,yes,Primary Injury,2021-03-12 03:35:51,Right,no,5f0592e2239906121f930929,2021-02-27,2021-03-12,2021-02-27,2021-03-12,1
2549,276.379347,0.847356,326.166780,Grass (Wet),2021-04-27 06:07:50,63.0,378.0,6.0,Practice (Field),5f0592e2239906121f930929,Kota Kawase,2021-04-27,2021-04-27,Competition,New Injury,2021-04-27 09:37:43,Competition - Domestic,TZ1,Acute,no,Primary Injury,2021-04-28 09:37:43,Left,no,5f0592e2239906121f930929,2021-04-27,2021-04-28,2021-04-27,2021-04-28,1
5297,0.000000,0.000000,0.000000,None,NaT,0.0,0.0,0.0,None,5f0592e2239906121f930929,NaN,2021-03-27,2021-03-27,Competition,New Injury,2021-03-27 04:11:17,Competition - Domestic,KZZ,Acute,no,Secondary Injury,2021-04-01 04:11:17,Right,no,5f0592e2239906121f930929,2021-03-27,2021-04-01,2021-03-27,2021-04-01,1
5299,0.000000,0.000000,0.000000,None,NaT,0.0,0.0,0.0,None,5f0592e2239906121f930929,NaN,2021-06-03,2021-06-03,Other,Recurrent Injury,2021-06-03 00:45:03,Other,NaN,Repetitive - Gradual Onset,yes,Primary Injury,2021-09-15 00:45:03,Right,yes,5f0592e2239906121f930929,2021-06-03,2021-09-15,2021-06-03,2021-09-15,1
5300,0.000000,0.000000,0.000000,None,NaT,0.0,0.0,0.0,None,5f0592e2239906121f930929,NaN,2021-08-04,2021-08-04,Other,Recurrent Injury,2021-08-04 00:52:10,Other,NaN,Repetitive - Gradual Onset,yes,Primary Injury,2021-08-04 00:52:10,Right,yes,5f0592e2239906121f930929,2021-08-04,2021-08-04,2021-08-04,2021-08-04,1
5301,0.000000,0.000000,0.000000,None,NaT,0.0,0.0,0.0,None,5f0592e2239906121f930929,NaN,2021-08-04,2021-08-04,Other,Recurrent Injury,2021-08-04 00:52:14,Other,NaN,Repetitive - Gradual Onset,yes,Primary Injury,2021-08-04 00:52:14,Right,yes,5f0592e2239906121f930929,2021-08-04,2021-08-04,2021-08-04,2021-08-04,1
5302,0.000000,0.000000,0.000000,None,NaT,0.0,0.0,0.0,None,5f0592e2239906121f930929,NaN,2021-08-04,2021-08-04,Other,Recurrent Injury,2021-08-04 00:52:31,Other,NaN,Repetitive - Gradual Onset,yes,Primary Injury,2021-08-04 00:52:31,Right,yes,5f0592e2239906121f930929,2021-08-04,2021-08-04,2021-08-04,2021-08-04,1
5303,0.000000,0.000000,0.000000,None,NaT,0.0,0.0,0.0,None,5f0592e2239906121f930929,NaN,2021-08-04,2021-08-04,Other,Recurrent Injury,2021-08-04 00:52:45,Other,NaN,Repetitive - Gradual Onset,yes,Primary Injury,2021-08-04 00:52:45,Right,yes,5f0592e2239906121f930929,2021-08-04,2021-08-04,2021-08-04,2021-08-04,1


In [22]:
outer_df.shape

(5305, 30)

In [23]:
# the number of injured records: 41
outer_df[outer_df["injury?"]==1].shape

(41, 30)

In [24]:
# the injured records
injured_exposure = outer_df[outer_df["injury?"]==1].sort_values(by=['formatteddate'])
injured_exposure

,acute,acwr,chronic,condition,date,duration,load,rpe,type,player_id,name,formatteddate,formatteddate_dt,Activity,Classification,Injury Date,Location,OSIICs,Onset,Past Injury,Reoccurance,Return Date,Side,Surgery,u_id,FormattedInjuryDate,FormattedReturnDate,FormattedInjuryDate_dt,FormattedReturnDate_dt,injury?
5293,0.000000,0.000000,0.000000,None,NaT,0.0,0.0,0.0,None,5f059355c37cc79a7d6a6caa,NaN,2020-11-02,2020-11-02,General Training,Recurrent Injury,2020-11-02 11:15:18,DTE - Domestic,GTX,Repetitive - Gradual Onset,yes,NaN,2021-02-16 11:15:18,Right,no,5f059355c37cc79a7d6a6caa,2020-11-02,2021-02-16,2020-11-02,2021-02-16,1
5294,0.000000,0.000000,0.000000,None,NaT,0.0,0.0,0.0,None,5f014aa4b8bec1cf08f7ca0f,NaN,2020-11-22,2020-11-22,General Training,New Injury,2020-11-22 21:58:31,DTE - Domestic,QMT,Repetitive - Gradual Onset,no,NaN,2020-12-22 21:58:31,Left,no,5f014aa4b8bec1cf08f7ca0f,2020-11-22,2020-12-22,2020-11-22,2020-12-22,1
1451,177.213568,0.757232,234.028062,Track,2020-12-21 04:28:04,50.0,200.0,4.0,Practice (Field),5f0593d9c37cc79a7d6a6cae,Shu Sasaki,2020-12-21,2020-12-21,General Training,New Injury,2020-12-21 22:01:30,DTE - Domestic,QM1,Repetitive - Sudden Onset,no,NaN,2021-01-25 22:01:30,Right,no,5f0593d9c37cc79a7d6a6cae,2020-12-21,2021-01-25,2020-12-21,2021-01-25,1
1724,174.747845,1.040819,167.894635,Grass (Dry),2021-01-25 05:08:03,60.0,300.0,5.0,Practice (Field),5f0593d9c37cc79a7d6a6cae,Shu Sasaki,2021-01-25,2021-01-25,General Training,Recurrent Injury,2021-01-25 22:06:39,DTE - Domestic,QM1,Repetitive - Sudden Onset,no,NaN,2021-02-07 22:06:39,Right,no,5f0593d9c37cc79a7d6a6cae,2021-01-25,2021-02-07,2021-01-25,2021-02-07,1
1779,230.509020,0.944906,243.949213,Grass (Dry),2021-02-01 05:11:18,60.0,360.0,6.0,Practice (Field),5f059326c37cc79a7d6a6ca9,james Ha,2021-02-01,2021-02-01,General Training,New Injury,2021-02-01 04:38:23,DTE - Domestic,KT9,Acute,no,Primary Injury,2021-03-05 04:38:23,Right,no,5f059326c37cc79a7d6a6ca9,2021-02-01,2021-03-05,2021-02-01,2021-03-05,1
2024,467.745603,1.269344,368.493849,Grass (Wet),2021-02-27 15:34:55,95.0,950.0,10.0,Game,5f0592e2239906121f930929,Kota Kawase,2021-02-27,2021-02-27,Competition,Recurrent Injury,2021-02-27 03:35:51,Competition - Domestic,AA1,Repetitive - Sudden Onset,yes,Primary Injury,2021-03-12 03:35:51,Right,no,5f0592e2239906121f930929,2021-02-27,2021-03-12,2021-02-27,2021-03-12,1
2124,165.890472,0.682486,243.068080,Artificial (Dry),2021-03-10 05:26:55,60.0,300.0,5.0,Modified,5f0593f1c37cc79a7d6a6caf,Stefan Pereira,2021-03-10,2021-03-10,Competition,New Injury,2021-03-10 08:48:53,Competition - Domestic,TM1,Acute,no,NaN,2021-03-10 08:48:53,Right,no,5f0593f1c37cc79a7d6a6caf,2021-03-10,2021-03-10,2021-03-10,2021-03-10,1
5295,0.000000,0.000000,0.000000,None,NaT,0.0,0.0,0.0,None,5f05939a239906121f930930,NaN,2021-03-17,2021-03-17,Competition,New Injury,2021-03-17 06:02:26,Competition - Domestic,ST2,Repetitive - Gradual Onset,no,Primary Injury,2021-03-27 06:02:26,Left,no,5f05939a239906121f930930,2021-03-17,2021-03-27,2021-03-17,2021-03-27,1
5296,0.000000,0.000000,0.000000,None,NaT,0.0,0.0,0.0,None,5f05939a239906121f930930,NaN,2021-03-19,2021-03-19,Competition,New Injury,2021-03-19 10:16:46,Competition - Domestic,NaN,Acute,no,Primary Injury,2021-07-30 10:16:46,Left,no,5f05939a239906121f930930,2021-03-19,2021-07-30,2021-03-19,2021-07-30,1
5297,0.000000,0.000000,0.000000,None,NaT,0.0,0.0,0.0,None,5f0592e2239906121f930929,NaN,2021-03-27,2021-03-27,Competition,New Injury,2021-03-27 04:11:17,Competition - Domestic,KZZ,Acute,no,Secondary Injury,2021-04-01 04:11:17,Right,no,5f0592e2239906121f930929,2021-03-27,2021-04-01,2021-03-27,2021-04-01,1


In [25]:
# the whole records (both uninjured and injured)
outer_df.sort_values(by=['formatteddate']).to_csv('exposure_injury_all_updated.csv', header=True, index=False, encoding = 'utf-8')

# Some players' exposure "type" is different from injury "activity" :

# Read the combined dataset and deal with anomalies

In [26]:
injury_exposure = pd.read_csv('exposure_injury_all_updated.csv')

In [27]:
injury_exposure.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5305 entries, 0 to 5304
Data columns (total 30 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   acute                   5305 non-null   float64
 1   acwr                    5305 non-null   float64
 2   chronic                 5305 non-null   float64
 3   condition               5305 non-null   object 
 4   date                    5293 non-null   object 
 5   duration                5305 non-null   float64
 6   load                    5305 non-null   float64
 7   rpe                     5305 non-null   float64
 8   type                    5305 non-null   object 
 9   player_id               5305 non-null   object 
 10  name                    5293 non-null   object 
 11  formatteddate           5305 non-null   object 
 12  formatteddate_dt        5305 non-null   object 
 13  Activity                40 non-null     object 
 14  Classification          41 non-null     

In [28]:
injury_exposure = injury_exposure.drop(columns = ["date", "formatteddate_dt","Injury Date", "Return Date",
                                                  "u_id", "FormattedInjuryDate_dt","FormattedReturnDate_dt"])
injury_exposure.rename(columns = {"Reoccurance":"Reoccurence"})

cols = ["formatteddate","FormattedInjuryDate","FormattedReturnDate"]
for col in cols:
    injury_exposure[col] = pd.to_datetime(injury_exposure[col])

In [29]:
injury_exposure.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5305 entries, 0 to 5304
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   acute                5305 non-null   float64       
 1   acwr                 5305 non-null   float64       
 2   chronic              5305 non-null   float64       
 3   condition            5305 non-null   object        
 4   duration             5305 non-null   float64       
 5   load                 5305 non-null   float64       
 6   rpe                  5305 non-null   float64       
 7   type                 5305 non-null   object        
 8   player_id            5305 non-null   object        
 9   name                 5293 non-null   object        
 10  formatteddate        5305 non-null   datetime64[ns]
 11  Activity             40 non-null     object        
 12  Classification       41 non-null     object        
 13  Location             39 non-null 

In [30]:
injury1_exposure = injury_exposure[injury_exposure["injury?"] == 1]

In [31]:
injury1_exposure.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41 entries, 1033 to 5160
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   acute                41 non-null     float64       
 1   acwr                 41 non-null     float64       
 2   chronic              41 non-null     float64       
 3   condition            41 non-null     object        
 4   duration             41 non-null     float64       
 5   load                 41 non-null     float64       
 6   rpe                  41 non-null     float64       
 7   type                 41 non-null     object        
 8   player_id            41 non-null     object        
 9   name                 29 non-null     object        
 10  formatteddate        41 non-null     datetime64[ns]
 11  Activity             40 non-null     object        
 12  Classification       41 non-null     object        
 13  Location             39 non-null

In [32]:
players_inj_lst = []
inj_players = set(injury1_exposure["player_id"])
for p in inj_players:
    df = injury_exposure[injury_exposure["player_id"] == p].reset_index()
    players_inj_lst.append(df)
   

In [33]:
inj_players

{'5f014aa4b8bec1cf08f7ca0f',
 '5f0592cc239906121f930928',
 '5f0592e2239906121f930929',
 '5f059326c37cc79a7d6a6ca9',
 '5f059355c37cc79a7d6a6caa',
 '5f059378239906121f93092e',
 '5f059388239906121f93092f',
 '5f05939a239906121f930930',
 '5f0593d9c37cc79a7d6a6cae',
 '5f0593f1c37cc79a7d6a6caf',
 '5f86fa4805bc5268fb4a00a5',
 '60f253c0854df5b9da1bac54',
 '60f25487854df5b9da1bac55',
 '6107d0a36f7b01ca9c0f6571',
 '61139a2f859c5efc737749ba',
 '61162b72840a022569af263d',
 '611a41e057d5db652811c0f3'}

In [34]:
def label_injury(row):
    if row["injury?"]==1:
        if (row["type"].startswith("Practice")) and (row["Activity"]=="General Training"):
            return False
        elif (row["type"]=="Game") and (row["Activity"]=="Competition"):
            return False
        else:
            return True
    else:
        return False
    
    
def posix_time(dt):
    return ((dt - datetime(1970, 1, 1)) / timedelta(seconds=1))/(24*60*60)

In [35]:
def player_handler(kota):
    kota['NeedActualInjuryDate'] = kota.apply(lambda row: label_injury(row), axis=1)
    kota['daydiff'] = kota['formatteddate'].apply(posix_time)

    kota_need = kota[kota["NeedActualInjuryDate"]==True]


    #activity_lst = ['Competition', 'General Training']
    #type_lst = ['Game', 'Practice']


    kota_activity_game_idx = kota_need.index[kota_need['Activity']=='Competition'].tolist()
    kota_activity_train_idx = kota_need.index[kota_need['Activity']=='General Training'].tolist()
    kota_activity_gt_idx = kota_activity_game_idx + kota_activity_train_idx
    kota_activity_other_idx = kota_need.index[(kota_need['Activity'] != 'Competition') & 
                                          (kota_need['Activity'] != 'General Training')].tolist()

    kota_type_game_idx = kota.index[kota['type']=='Game'].tolist()
    kota_type_train_idx = kota.index[kota['type'].apply(lambda row: row.startswith('Practice'))].tolist()

    res_lst = []
    for idx in kota_activity_game_idx:
        for idx2 in kota_type_game_idx:
            if idx2 <= idx:
                res = idx2
        res_lst.append(res)
    #print(res_lst)
    for idx in kota_activity_train_idx:
        for idx2 in kota_type_train_idx:
            if idx2 <= idx:
                res = idx2
        res_lst.append(res)
    #print(res_lst)

    
    
    actual_injury_dt = kota['FormattedInjuryDate'].tolist()

    for i in range(len(kota_activity_gt_idx)):
        diff = kota.loc[kota_activity_gt_idx[i]]['daydiff'] - kota.loc[res_lst[i]]['daydiff']
        if diff <= 7:
            actual_injury_dt[kota_activity_gt_idx[i]] = kota.loc[res_lst[i]]['formatteddate']
        else:
            j = kota_activity_gt_idx[i]-1
            #print(j)
            while (kota.loc[j]['type'] == 'Rehab') or (kota.loc[j]['type'] == 'Modified') or (kota.loc[j]['formatteddate'] == kota[kota['type']=='Rehab']['formatteddate'].tolist()[0]):
                j = j - 1 
                #print(j)
            actual_injury_dt[kota_activity_gt_idx[i]] = kota.loc[j]['formatteddate']
        
    for i in range(len(kota_activity_other_idx)):
        j = kota_activity_other_idx[i]-1
        while (kota.loc[j]['type'] == 'Rehab') or (kota.loc[j]['type'] == 'Modified') or (kota.loc[j]['formatteddate'] == kota[kota['type']=='Rehab']['formatteddate'].tolist()[0]):
            j = j - 1
        actual_injury_dt[kota_activity_other_idx[i]] = kota.loc[j]['formatteddate']
               
        
    kota['actualInjuryDate'] = actual_injury_dt
    return kota

In [36]:
players_inj_lst = []
inj_players = set(injury_exposure["player_id"])
for p in inj_players:
    df = injury_exposure[injury_exposure["player_id"] == p].reset_index()
    df_new = player_handler(df)
    players_inj_lst.append(df_new)

new_players_inj = pd.concat(players_inj_lst)

In [37]:
new_players_inj

,index,acute,acwr,chronic,condition,duration,load,rpe,type,player_id,name,formatteddate,Activity,Classification,Location,OSIICs,Onset,Past Injury,Reoccurance,Side,Surgery,FormattedInjuryDate,FormattedReturnDate,injury?,NeedActualInjuryDate,daydiff,actualInjuryDate
0,4,390.000000,1.000000,390.000000,Artificial (Dry),65.0,390.0,6.0,Practice (Field),5f059378239906121f93092e,Xavi Cheng,2020-06-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,0,False,18442.0,NaT
1,10,481.250000,1.159157,415.172414,Grass (Dry),55.0,755.0,8.0,Practice (Field),5f059378239906121f93092e,Xavi Cheng,2020-06-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,0,False,18443.0,NaT
2,18,371.250000,0.964718,384.827586,Gym/Indoors,45.0,315.0,7.0,Practice (Gym),5f059378239906121f93092e,Xavi Cheng,2020-06-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,0,False,18443.0,NaT
3,34,328.437500,0.882705,372.080856,Artificial (Dry),40.0,200.0,5.0,Modified,5f059378239906121f93092e,Xavi Cheng,2020-07-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,0,False,18444.0,NaT
4,40,418.828125,1.062999,394.006314,Artificial (Dry),60.0,690.0,7.0,Practice (Field),5f059378239906121f93092e,Xavi Cheng,2020-07-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,0,False,18445.0,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143,5238,279.680901,1.182060,236.604740,Sand,75.0,375.0,5.0,Practice (Beach),60f254d8a1b3ec106cc401da,Khan Jahangir,2022-01-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,0,False,19018.0,NaT
144,5248,284.760676,1.181693,240.976827,Track,50.0,300.0,6.0,Quarantine - Run,60f254d8a1b3ec106cc401da,Khan Jahangir,2022-01-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,0,False,19019.0,NaT
145,5273,303.570507,1.218252,249.185322,Sand,60.0,360.0,6.0,Practice (Beach),60f254d8a1b3ec106cc401da,Khan Jahangir,2022-01-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,0,False,19020.0,NaT
146,5283,183.068807,0.846459,216.275972,Sand,55.0,220.0,4.0,Practice (Beach),60f254d8a1b3ec106cc401da,Khan Jahangir,2022-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,0,False,19023.0,NaT


In [38]:
new_players_inj[new_players_inj['NeedActualInjuryDate']==1]

,index,acute,acwr,chronic,condition,duration,load,rpe,type,player_id,name,formatteddate,Activity,Classification,Location,OSIICs,Onset,Past Injury,Reoccurance,Side,Surgery,FormattedInjuryDate,FormattedReturnDate,injury?,NeedActualInjuryDate,daydiff,actualInjuryDate
299,4434,0.000000,0.000000,0.000000,None,0.0,0.0,0.0,None,5f059378239906121f93092e,NaN,2021-11-29,General Training,Recurrent Injury,DTE - Domestic,NaN,Acute,no,Primary Injury,Right,no,2021-11-29,2021-12-01,1,True,18960.0,2021-11-26
178,2187,0.000000,0.000000,0.000000,None,0.0,0.0,0.0,None,5f05939a239906121f930930,NaN,2021-03-17,Competition,New Injury,Competition - Domestic,ST2,Repetitive - Gradual Onset,no,Primary Injury,Left,no,2021-03-17,2021-03-27,1,True,18703.0,2021-03-13
179,2200,0.000000,0.000000,0.000000,None,0.0,0.0,0.0,None,5f05939a239906121f930930,NaN,2021-03-19,Competition,New Injury,Competition - Domestic,NaN,Acute,no,Primary Injury,Left,no,2021-03-19,2021-07-30,1,True,18705.0,2021-03-13
124,1033,0.000000,0.000000,0.000000,None,0.0,0.0,0.0,None,5f059355c37cc79a7d6a6caa,NaN,2020-11-02,General Training,Recurrent Injury,DTE - Domestic,GTX,Repetitive - Gradual Onset,yes,NaN,Right,no,2020-11-02,2021-02-16,1,True,18568.0,2020-10-31
21,3299,319.629772,0.882865,362.036919,Grass (Dry),25.0,125.0,5.0,Game,61139a2f859c5efc737749ba,Ju Yingzhi,2021-09-10,NaN,New Injury,NaN,NaN,Acute,no,NaN,NaN,no,2021-09-10,2021-09-10,1,True,18880.0,2021-09-09
220,2278,0.000000,0.000000,0.000000,None,0.0,0.0,0.0,None,5f0592e2239906121f930929,NaN,2021-03-27,Competition,New Injury,Competition - Domestic,KZZ,Acute,no,Secondary Injury,Right,no,2021-03-27,2021-04-01,1,True,18713.0,2021-03-21
248,2560,276.379347,0.847356,326.166780,Grass (Wet),63.0,378.0,6.0,Practice (Field),5f0592e2239906121f930929,Kota Kawase,2021-04-27,Competition,New Injury,Competition - Domestic,TZ1,Acute,no,Primary Injury,Left,no,2021-04-27,2021-04-28,1,True,18744.0,2021-04-23
275,2835,0.000000,0.000000,0.000000,None,0.0,0.0,0.0,None,5f0592e2239906121f930929,NaN,2021-06-03,Other,Recurrent Injury,Other,NaN,Repetitive - Gradual Onset,yes,Primary Injury,Right,yes,2021-06-03,2021-09-15,1,True,18781.0,2021-05-30
276,2866,0.000000,0.000000,0.000000,None,0.0,0.0,0.0,None,5f0592e2239906121f930929,NaN,2021-08-04,Other,Recurrent Injury,Other,NaN,Repetitive - Gradual Onset,yes,Primary Injury,Right,yes,2021-08-04,2021-08-04,1,True,18843.0,2021-06-03
277,2867,0.000000,0.000000,0.000000,None,0.0,0.0,0.0,None,5f0592e2239906121f930929,NaN,2021-08-04,Other,Recurrent Injury,Other,NaN,Repetitive - Gradual Onset,yes,Primary Injury,Right,yes,2021-08-04,2021-08-04,1,True,18843.0,2021-08-04


In [39]:
new_players_inj[~((new_players_inj.duplicated(['player_id','formatteddate'], keep='first'))
                &(new_players_inj['injury?']==1)
               &(new_players_inj['player_id']=='5f0592e2239906121f930929'))]

,index,acute,acwr,chronic,condition,duration,load,rpe,type,player_id,name,formatteddate,Activity,Classification,Location,OSIICs,Onset,Past Injury,Reoccurance,Side,Surgery,FormattedInjuryDate,FormattedReturnDate,injury?,NeedActualInjuryDate,daydiff,actualInjuryDate
0,4,390.000000,1.000000,390.000000,Artificial (Dry),65.0,390.0,6.0,Practice (Field),5f059378239906121f93092e,Xavi Cheng,2020-06-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,0,False,18442.0,NaT
1,10,481.250000,1.159157,415.172414,Grass (Dry),55.0,755.0,8.0,Practice (Field),5f059378239906121f93092e,Xavi Cheng,2020-06-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,0,False,18443.0,NaT
2,18,371.250000,0.964718,384.827586,Gym/Indoors,45.0,315.0,7.0,Practice (Gym),5f059378239906121f93092e,Xavi Cheng,2020-06-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,0,False,18443.0,NaT
3,34,328.437500,0.882705,372.080856,Artificial (Dry),40.0,200.0,5.0,Modified,5f059378239906121f93092e,Xavi Cheng,2020-07-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,0,False,18444.0,NaT
4,40,418.828125,1.062999,394.006314,Artificial (Dry),60.0,690.0,7.0,Practice (Field),5f059378239906121f93092e,Xavi Cheng,2020-07-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,0,False,18445.0,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143,5238,279.680901,1.182060,236.604740,Sand,75.0,375.0,5.0,Practice (Beach),60f254d8a1b3ec106cc401da,Khan Jahangir,2022-01-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,0,False,19018.0,NaT
144,5248,284.760676,1.181693,240.976827,Track,50.0,300.0,6.0,Quarantine - Run,60f254d8a1b3ec106cc401da,Khan Jahangir,2022-01-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,0,False,19019.0,NaT
145,5273,303.570507,1.218252,249.185322,Sand,60.0,360.0,6.0,Practice (Beach),60f254d8a1b3ec106cc401da,Khan Jahangir,2022-01-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,0,False,19020.0,NaT
146,5283,183.068807,0.846459,216.275972,Sand,55.0,220.0,4.0,Practice (Beach),60f254d8a1b3ec106cc401da,Khan Jahangir,2022-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,0,False,19023.0,NaT


In [40]:
# deal with the special cases
new_players_inj = new_players_inj[~((new_players_inj.duplicated(['player_id','formatteddate'], keep='first'))
                &(new_players_inj['injury?']==1)
               &(new_players_inj['player_id']=='5f0592e2239906121f930929'))]

new_players_inj.at[new_players_inj.index[(new_players_inj['player_id']=='5f0592e2239906121f930929')
                &(new_players_inj['FormattedInjuryDate']=='2021-08-04')].tolist()[0], 'actualInjuryDate'] =  '2021-05-30'


#new_players_inj[new_players_inj['NeedActualInjuryDate']==1]

In [41]:
new_players_inj[new_players_inj['injury?']==1]
#length = 25

,index,acute,acwr,chronic,condition,duration,load,rpe,type,player_id,name,formatteddate,Activity,Classification,Location,OSIICs,Onset,Past Injury,Reoccurance,Side,Surgery,FormattedInjuryDate,FormattedReturnDate,injury?,NeedActualInjuryDate,daydiff,actualInjuryDate
299,4434,0.000000,0.000000,0.000000,None,0.0,0.0,0.0,None,5f059378239906121f93092e,NaN,2021-11-29,General Training,Recurrent Injury,DTE - Domestic,NaN,Acute,no,Primary Injury,Right,no,2021-11-29,2021-12-01,1,True,18960.0,2021-11-26
25,3678,128.344677,0.972797,131.933664,Grass (Wet),12.0,84.0,7.0,Game,61162b72840a022569af263d,Ondy So,2021-10-14,Competition,New Injury,Competition - Domestic,NaN,Acute,no,Primary Injury,Right,no,2021-10-14,2021-10-25,1,False,18914.0,2021-10-14
178,2187,0.000000,0.000000,0.000000,None,0.0,0.0,0.0,None,5f05939a239906121f930930,NaN,2021-03-17,Competition,New Injury,Competition - Domestic,ST2,Repetitive - Gradual Onset,no,Primary Injury,Left,no,2021-03-17,2021-03-27,1,True,18703.0,2021-03-13
179,2200,0.000000,0.000000,0.000000,None,0.0,0.0,0.0,None,5f05939a239906121f930930,NaN,2021-03-19,Competition,New Injury,Competition - Domestic,NaN,Acute,no,Primary Injury,Left,no,2021-03-19,2021-07-30,1,True,18705.0,2021-03-13
251,3124,672.188729,1.574461,426.932707,Grass (Wet),115.0,805.0,7.0,Practice (Field),5f05939a239906121f930930,Him Tse,2021-08-26,General Training,New Injury,DTE - Domestic,NaN,Acute,no,Primary Injury,Right,no,2021-08-26,2021-09-16,1,False,18865.0,2021-08-26
124,1033,0.000000,0.000000,0.000000,None,0.0,0.0,0.0,None,5f059355c37cc79a7d6a6caa,NaN,2020-11-02,General Training,Recurrent Injury,DTE - Domestic,GTX,Repetitive - Gradual Onset,yes,NaN,Right,no,2020-11-02,2021-02-16,1,True,18568.0,2020-10-31
285,2718,473.102208,1.364938,346.610714,Grass (Dry),95.0,950.0,10.0,Game,5f059355c37cc79a7d6a6caa,Jack Sealy,2021-05-16,Competition,New Injury,Competition - Domestic,TM1,Acute,no,Primary Injury,Left,no,2021-05-16,2021-05-20,1,False,18763.0,2021-05-16
298,2862,197.812503,3.159957,62.599740,Grass (Wet),85.0,510.0,6.0,Practice (Field),5f059355c37cc79a7d6a6caa,Jack Sealy,2021-08-03,General Training,New Injury,NaN,HV1,Acute,no,NaN,Right,no,2021-08-03,2021-08-03,1,False,18842.0,2021-08-03
299,2865,197.812503,3.159957,62.599740,Grass (Wet),85.0,510.0,6.0,Practice (Field),5f059355c37cc79a7d6a6caa,Jack Sealy,2021-08-03,General Training,New Injury,DTE - Domestic,NaN,Acute,no,Primary Injury,Right,no,2021-08-03,2021-08-25,1,False,18842.0,2021-08-03
21,3299,319.629772,0.882865,362.036919,Grass (Dry),25.0,125.0,5.0,Game,61139a2f859c5efc737749ba,Ju Yingzhi,2021-09-10,NaN,New Injury,NaN,NaN,Acute,no,NaN,NaN,no,2021-09-10,2021-09-10,1,True,18880.0,2021-09-09


In [42]:
player = set(new_players_inj['player_id'])
print(player)
print(len(player))
injured_player = set(new_players_inj[new_players_inj['injury?']==1]['player_id'])
print(injured_player)

{'5f059378239906121f93092e', '60f254fb854df5b9da1bac56', '61162b72840a022569af263d', '5f05939a239906121f930930', '5f059355c37cc79a7d6a6caa', '61139a2f859c5efc737749ba', '611a41e057d5db652811c0f3', '60f25487854df5b9da1bac55', '5f059388239906121f93092f', '5f059326c37cc79a7d6a6ca9', '5f0592e2239906121f930929', '6153f4f14983cc4b427f2af2', '60f254ada1b3ec106cc401d9', '5f86fa4805bc5268fb4a00a5', '5f0593f1c37cc79a7d6a6caf', '5f014aa4b8bec1cf08f7ca0f', '6107d0a36f7b01ca9c0f6571', '60f253c0854df5b9da1bac54', '5f0593d9c37cc79a7d6a6cae', '5f0592cc239906121f930928', '60f254d8a1b3ec106cc401da'}
21
{'5f059378239906121f93092e', '6107d0a36f7b01ca9c0f6571', '61139a2f859c5efc737749ba', '5f059355c37cc79a7d6a6caa', '5f86fa4805bc5268fb4a00a5', '611a41e057d5db652811c0f3', '60f253c0854df5b9da1bac54', '60f25487854df5b9da1bac55', '5f059388239906121f93092f', '5f0593f1c37cc79a7d6a6caf', '5f0593d9c37cc79a7d6a6cae', '5f0592cc239906121f930928', '61162b72840a022569af263d', '5f059326c37cc79a7d6a6ca9', '5f0592e2239906

In [43]:
new_players_inj.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5302 entries, 0 to 147
Data columns (total 27 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   index                 5302 non-null   int64         
 1   acute                 5302 non-null   float64       
 2   acwr                  5302 non-null   float64       
 3   chronic               5302 non-null   float64       
 4   condition             5302 non-null   object        
 5   duration              5302 non-null   float64       
 6   load                  5302 non-null   float64       
 7   rpe                   5302 non-null   float64       
 8   type                  5302 non-null   object        
 9   player_id             5302 non-null   object        
 10  name                  5293 non-null   object        
 11  formatteddate         5302 non-null   datetime64[ns]
 12  Activity              37 non-null     object        
 13  Classification     

In [44]:
new_players_inj.to_csv('cleaned_player_inj_updated.csv', header=True, index=False, encoding = 'utf-8')